# Regression Tests

In [1]:
import tprstats
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
coffee_data = pd.read_excel('../data/Coffee_Data.xlsx', engine='openpyxl')
diamonds_211 = pd.read_excel('../data/Diamonds_211.xlsx', engine='openpyxl')
solar_data = pd.read_excel('../data/Solar_Data.xlsx', engine='openpyxl')
solar_new_prospects = pd.read_excel('../data/Solar_New_Prospects.xlsx', engine='openpyxl')

## Wald Tests
### Linear Models
#### Cross-section aka coefTestH

In [2]:
model_diamonds = tprstats.model("cs", 'Price ~ Carat + C(Clarity)', diamonds_211)
model_diamonds.wald_test('C(Clarity)[T.5] = C(Clarity)[T.6]')

p-value:  0.2434


#### Time Series Wald Test aka coefTestHAC

In [3]:
model_coffee_ts = tprstats.model(name = "ts", formula='cons~rpcarb+rpcinc+rpcofe', data=coffee_data)
model_coffee_ts.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cons   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.953
No. Observations:                  61   F-statistic:                     207.9
Covariance Type:                  HAC   Prob (F-statistic):           1.18e-30
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0576      1.839      0.031      0.975      -3.626       3.741
rpcarb         0.0901      0.008     11.549      0.000       0.074       0.106
rpcinc         0.1235      0.033      3.762      0.000       0.058       0.189
rpcofe        -0.0126      0.004     -2.835      0.006      -0.021      -0.004
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 2 lags and with small sample correction
[2] The condition number is large, 3.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
model_coffee_ts.wald_test('rpcofe=-0.013, rpcarb=0.08')

p-value:  0.4026


### Binary Choice Wald Tests 
#### Logit Wald Test

In [5]:
solar_logit=tprstats.model("logit",formula="Sale ~ AnnualSave + AnnualCost + DrivesHybrid + NearbySolar + HomeValue",data=solar_data)

Optimization terminated successfully.
         Current function value: 0.102926
         Iterations 9


In [6]:
solar_logit.wald_test('AnnualSave=4 , AnnualCost=-3')

/workspaces/tprstats-python/.pixi/envs/default/lib/python3.12/site-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[0.67140993]], p-value=0.7148339750029686, df_denom=2>

#### Probit Wald Test

In [7]:
solar_probit=tprstats.model("probit",formula="Sale ~ AnnualSave + AnnualCost + DrivesHybrid + NearbySolar + HomeValue",data=solar_data)
print(solar_probit.summary())

Optimization terminated successfully.
         Current function value: 0.100579
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                   Sale   No. Observations:                 1500
Model:                         Probit   Df Residuals:                     1494
Method:                           MLE   Df Model:                            5
Date:                Mon, 27 Jan 2025   Pseudo R-squ.:                  0.3010
Time:                        01:26:25   Log-Likelihood:                -150.87
converged:                       True   LL-Null:                       -215.84
Covariance Type:            nonrobust   LLR p-value:                 2.450e-26
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -6.5365      2.307     -2.833      0.005     -11.058      -2.015
AnnualSave       1.9807

In [8]:
solar_probit.wald_test('AnnualSave=2.3 , AnnualCost=-1.9')

/workspaces/tprstats-python/.pixi/envs/default/lib/python3.12/site-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[1.51168264]], p-value=0.4696153508770117, df_denom=2>

## Ramsey RESET
Test of functional form for linear models. 

In [9]:
print(model_coffee_ts.ramsey_test())


   power    pvalue
0      2  0.182315
1      3  0.372575
